In [ ]:
## Importing neccessary libraries

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:


# Initial model testing data 
# data = [
#     {'english': 'I am fine, and you?', 'akan': 'Me ho ye'},
#     {'english': 'Good morning', 'akan': 'Mema wo akye'},
#     {'english': 'Thank you', 'akan': 'Medase'},
#     {'english': 'What is your name?', 'akan': 'Wo din de sen?'},
# ]
# current data - 25,000 parallel sentences
data = pd.read_json('dataset.json')

# Extracting sentences into respective variables
english_texts = [item['english'] for item in data]
akan_texts = [item['akan'] for item in data]

# English Sentences Tokenization
eng_tokenizer = Tokenizer(filters='')
eng_tokenizer.fit_on_texts(english_texts)
eng_sequences = eng_tokenizer.texts_to_sequences(english_texts)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_padded = pad_sequences(eng_sequences, padding='post')

# Akan Sentences Tokenization
akan_tokenizer = Tokenizer(filters='')
akan_tokenizer.fit_on_texts(akan_texts)
akan_sequences = akan_tokenizer.texts_to_sequences(akan_texts)
akan_vocab_size = len(akan_tokenizer.word_index) + 1
akan_padded = pad_sequences(akan_sequences, padding='post')

# Training, Testing & Validation Split
X_train, X_test, y_train, y_test = train_test_split(eng_padded, akan_padded, test_size=0.2, random_state=42)

# Model Parameters
embedding_dim = 64
units = 128

# Model Layers (Encoder-Decoder Layers) - return_sequences and return_state
encoder_inputs = tf.keras.Input(shape=(None,))
encoder_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_gru = GRU(units, return_sequences=False, return_state=True)
encoder_outputs, encoder_state = encoder_gru(encoder_embedding)  # Ensure encoder_state is returned

decoder_inputs = tf.keras.Input(shape=(None,))
decoder_embedding = Embedding(akan_vocab_size, embedding_dim)(decoder_inputs)
decoder_gru = GRU(units, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=encoder_state)  # Pass encoder_state
decoder_dense = Dense(akan_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compiling Model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Training
model.fit([X_train, y_train[:, :-1]], y_train[:, 1:], epochs=50, batch_size=32, validation_data=([X_test, y_test[:, :-1]], y_test[:, 1:]))

# Saving Model
model.save('english_to_akan_gru.h5')




Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 2.4014 - accuracy: 0.0000e+00 - val_loss: 2.3926 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 67ms/step - loss: 2.3793 - accuracy: 0.5556 - val_loss: 2.3916 - val_accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 61ms/step - loss: 2.3574 - accuracy: 0.4444 - val_loss: 2.3905 - val_accuracy: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 44ms/step - loss: 2.3350 - accuracy: 0.4444 - val_loss: 2.3893 - val_accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 47ms/step - loss: 2.3117 - accuracy: 0.4444 - val_loss: 2.3879 - val_accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 64ms/step - loss: 2.2868 - accuracy: 0.4444 - val_loss: 2.3862 - val_accuracy: 0.3333
Epoch 7/50
1/1 [==============================] - 0s 80ms/step - loss: 2.2598 - accuracy: 0.4444 - val_loss: 2.3842 - val_accuracy: 0.3333
Epoch 8/50
1/1 [=====

In [ ]:
# Model Summary - To be sure the layers are arranged in the exact order we want it
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, None, 64)     896         ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, None, 64)     704         ['input_6[0][0]']                
                                                                                            

#### Model Inference 

In [ ]:


def load_encoder_decoder(model, units):
    """Extract encoder and decoder models for inference."""
    
    # Encoder Layer
    encoder_inputs = model.input[0]  # English input
    encoder_embedding = model.layers[1](encoder_inputs)
    encoder_outputs, encoder_state = model.layers[2](encoder_embedding)
    encoder_model = tf.keras.Model(encoder_inputs, encoder_state)

    # Decoder Layer
    decoder_inputs = tf.keras.Input(shape=(None,))
    decoder_state_input = tf.keras.Input(shape=(units,))

    decoder_embedding = model.layers[3](decoder_inputs)
    decoder_gru = model.layers[4]
    
    decoder_outputs, decoder_state = decoder_gru(decoder_embedding, initial_state=decoder_state_input)
    decoder_dense = model.layers[5]
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = tf.keras.Model([decoder_inputs, decoder_state_input], [decoder_outputs, decoder_state])

    return encoder_model, decoder_model

def preprocess_sentence(sentence, tokenizer, max_len):
    sequence = tokenizer.texts_to_sequences([sentence])
    return pad_sequences(sequence, maxlen=max_len, padding='post')

def translate_sentence(encoder_model, decoder_model, sentence, eng_tokenizer, akan_tokenizer, max_len):    
    #  Input Preprocesing
    encoder_input = preprocess_sentence(sentence, eng_tokenizer, max_len)

    # Encoding input sentence
    encoder_state = encoder_model.predict(encoder_input)

    # Start decoding with "<start>" token if applicable, else use 1
    decoder_input = np.zeros((1, 1))
    decoder_input[0, 0] = akan_tokenizer.word_index.get('<start>', 1)

    decoded_sentence = []

    for _ in range(max_len):
        output_tokens, state = decoder_model.predict([decoder_input, encoder_state])
        predicted_id = np.argmax(output_tokens[0, -1, :])
        
        # Stop at <end> token
        if predicted_id == 0:  
            break
        
        word = akan_tokenizer.index_word.get(predicted_id, '')
        decoded_sentence.append(word)
        
        # Updating decoder input for next iteration
        decoder_input[0, 0] = predicted_id
        
        # Updating the state
        encoder_state = state  
    
    return ' '.join(decoded_sentence)

# Loading the trained model
model = tf.keras.models.load_model('english_to_akan_gru.h5')

# Extracting encoder and decoder
encoder_model, decoder_model = load_encoder_decoder(model, units=128)

# Test translation
english_sentence = "What is your name?"
akan_translation = translate_sentence(encoder_model, decoder_model, english_sentence, eng_tokenizer, akan_tokenizer, max_len=10)

print(f"English: {english_sentence}")
print(f"Akan: {akan_translation}")
